<h1 style="color:#333333; text-align:center; line-height: 0;"> <img style="right;" src="logo.png" width=18% height=18%> Reinforcement Learning | Assignment 4 
</h1>
<br/><br/>


The goal of this assignment is to implement:

- Actor
- Critic model


___Total points:___ 100

<h2 style="color:#A7BD3F;"> Section 1: Introduction to model predictive control (MPC) </h2>

***
###  <font color="blue"> 1.1 Intuition behind MPC </font>
**MPC** is one of the most popular methods of obtaining the optimal controller and, in fact, is an industry standard.
Let us start from the most reasonable question. Why should we care about **MPC**? Why do we even need that thing? Are there any problems that naturally lead to the development of such a model?

In fact, yes. There are infinitely many applications of **MPC**, including petroleum extraction, agricultural facilities, automatic assembly and so on. But let us take a look at the development of such a type of machines that are impressive not only to an expert in the field - humanoid robots. If you've ever seen a video of Boston Dynamics' humanoid robots, you've probably wondered, how do the engineers made it work? How does such a complex machine make those precise movements so that it is capable of performing acrobatics that already exceeds the abilities of a typical human in some aspects (i.e. backflip)?

The answer is, you guessed it, **MPC**. 

The performance of the modern robots comes with its cost. The problem of generating control for any robot requires:

- real time performance, as there are in general almost no stable states in the robot's movement, and all the calculations should be performed quickly
- optimizing a complex composite cost to a certain time horizon in order to follow the high-level plan
- (in some cases) discrete-continuous optimization, which is difficult
- taking into account various types of constraints, i.e.
    1. torque and angle limits for the servomotors (in a form of inequality)
    2. functional constraints following from the problem statement, that do not always allow for an analytical solution 
    3. also there could be constraints on foot placement, body placement, slippery surfaces, etc.
    4. constraints of the limbs non-intersection (if this is a case)

All in all, such a problem could lack a closed form solution, like $\boldsymbol u = \boldsymbol f(\text{state}, \text{target})$.

One of the most fundamental important ways to obtain an optimal controller is __MPC__. 

* __MPC-like algorithms__ are ones of the few that can handle very complex constraints, including functional, nonlinear, nonconvex, etc.

Informally speaking, algorithms that generate control for a complex dynamical system typically have to be predictive. And taking the system model into account helps along the way. In order to optimize an objective along the trajectory, we could try to estimate the system behaviour in the future. This is litearlly what **MPC** does. With that in hand, let us state the problem and the **MPC** more formally.

###  <font color="blue"> 1.2 MPC mathematical description </font>
<a id='2.2'></a>

First of all, we consider a controlled physical system described by the system of ordinary differential equations 
<a id='System'></a>
\begin{equation}
\begin{cases}
\dot{\boldsymbol x} = \boldsymbol f(\boldsymbol x, \boldsymbol u)\\
\boldsymbol y = \boldsymbol h(\boldsymbol x) \\
\boldsymbol x(0)=\boldsymbol x_{0}\\
\end{cases}
\end{equation} where $x_{0}$ is the **initial state**.In the following table we introduce some basic notation. From now on, we will write vectors in **bold**.
<a id='Notation'></a>

| Notation &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;| &nbsp;&nbsp;Description |
|:-----------------------:|-------------|
| $\boldsymbol f(\cdot, \cdot, \cdot) : \mathbb{R}^{n+1}\times \mathbb{R}^{m} \rightarrow \mathbb{R}^{n}$ |A **state dynamic function** or, more informally, **righ-hand-side** of a system <br /> of ordinary differential equations $\dot{\boldsymbol x} = \boldsymbol f(t, \boldsymbol x, \boldsymbol u)$|
| $\boldsymbol x \in \mathbb{R}^{n} $ | An element of the **state space** of a controlled system of dimensionality $n$ |
| $\boldsymbol u \in \mathbb{R}^{m}$ | An element of the **action space** of a controlled system of dimensionality $m$ |
| $\boldsymbol y \in \mathbb{R}^{k}$ | An **observartion**|
| $\mathbb{X}\subset \mathbb{R}^{n} $| **State constraint set**|
| $\mathbb{U}\subset \mathbb{R}^{m} $| **Action constraint set**|
| $\boldsymbol h(\cdot): \mathbb{R}^{n} \rightarrow \mathbb{R}^{k}$ | **Observation function**  |
| $\boldsymbol\rho(\cdot) : \mathbb{R}^{n} \rightarrow \mathbb{R}^{m}$ | **Policy** function |
| $r(\cdot) : \mathbb{R}^n \times \mathbb{R}^m \rightarrow \mathbb{R}$ | **Running cost** function  |


Here we would like to make a few clarifications on the notation introduced.
* For **MPC** N-step cost has the following form  $J_N(\boldsymbol x_{k}, \kappa(\cdot)):=\int_{k\delta}^{(k+N)\delta}\rho(\boldsymbol x(t), \kappa(\boldsymbol x(t))) dt$
* In the following problems we consider $\mathbb{X} = \mathbb{R}^{n}$ without any remarks but in general we might want to introduce some reasonable **state constraint set**.
* When working with the system, we cannot know the exact value of the characteristics we are interested in. What we do is **make a measurement** calculatung **observation function** $\boldsymbol h(\cdot)$. The result we call an **observation** $\boldsymbol y$. For now we consider $\boldsymbol y = \boldsymbol h(\boldsymbol x) := \boldsymbol x$ further but in general it might not be so! For example, if we control a body motion on a plane $(x,y)$, we could measure a distance to the origin: $\boldsymbol h(\boldsymbol x) = \lVert \boldsymbol x \rVert$. 
* Most modern controllers are digital, therefore the control signal is generated by sampling with some **sampling time** $\delta$. In that case we call it **digital control setting**. The following table gives a comparison between mathematical description of the original setting of the controlled system and its **digital control setting**.
<a id='Comparison_table'></a>
|  Original setting | Digital control setting |
|:-----------------------|:--------------------------|
|1) $ \qquad \dot{\boldsymbol x}=\boldsymbol f\left(\boldsymbol x, \boldsymbol u\right) $ | $\text{1*)} \quad \dot{\boldsymbol x}=\boldsymbol f\left(\boldsymbol x, \boldsymbol u^{\delta}\right)$|
| $\text{2)} \quad \boldsymbol u=\boldsymbol u(t), t \in[0, T]$  | $\text{2*)} \quad \boldsymbol u_k(t)=\boldsymbol u(k\delta) , t \in[k \delta,(k+1) \delta]$  | 
| $$\text{3)} \quad \boldsymbol x(t):=\boldsymbol x_{t_0}+\int_{t_0}^{t}f\left(\boldsymbol x(\tau), \boldsymbol u(\tau) \right) d \tau$$| $$\text{3*)} \quad \boldsymbol x^{\boldsymbol u_{k}}(t):=\boldsymbol x_{k}+\int_{k \delta}^{t}f\left(\boldsymbol x(\tau), \boldsymbol u_{k}\right) d \tau$$|     

<!--- $$\boldsymbol x_{i \mid k}:=\boldsymbol x((k+i-1) \delta), k \in \mathbb{N}$$
$\boldsymbol u^{\delta}(t) \equiv \boldsymbol u_{i \mid k}=\kappa\left(\boldsymbol x_{i \mid k}\right), t \in[k \delta,(k+i) \delta]$ 
One can see that for any $k \in \mathbb{N}$, the state $x^{u_{k}}(t)$ at $t \geq k \delta$ under $u_{k}$ satisfies
-->
<p style="text-align: center;">

</p>

From now and on by the system we mean a system in the **digital control setting**, which is illustrated on the figure below .
<img src="digital_control_setting.svg" width=40% height=40% />

**Objective:**

In general, our optimal control problem is written as $\min_{\kappa(\cdot)}\int_{k\delta}^{(k+N)\delta}\rho(\boldsymbol x(t), \kappa(\boldsymbol x(t))) dt$, where $N$ is **prediction horizon**. But in **MPC** we switch to a discrete sum instead of an integral: $$\min _{\boldsymbol u_{k + i}: i=0, \ldots, N-1} \left(\sum_{i=1}^{N-1} \rho \left(\hat{\boldsymbol x}_{k + i}, \boldsymbol u_{k + i}\right)\delta \right) $$ Wich is obviously equivalent to the following form:
$$\min _{ \boldsymbol u_{k + i}: i=0, \ldots, N-1} \left(\sum_{i=1}^{N-1} \rho \left(\hat{\boldsymbol x}_{k + i}, \boldsymbol u_{k + i}\right)\right)$$  
And now we see that it fits conviniently in our **digital control setting**: we will just find a minimizing sequence of actions for our digital model predictive control for stage costs predicted for N steps forward.  Notice that under digital control our state evolves according to [3*)](#Comparison_table) . Now we need to somehow numerically evaluate the interal $\int_{k \delta}^{t}f\left(\boldsymbol x(\tau), \boldsymbol u_{k}\right) d \tau$. In order to do this one might use any numerical integration scheme. For example the **Euler scheme** :
<a id='Euler'></a>
$$
\boldsymbol x_{k+1}=\boldsymbol x_{k}+\delta \boldsymbol f\left(\boldsymbol x_{k}, \boldsymbol u_{k}\right) \text {, }
$$
#### Algorithm <sup>[1]</sup>:
<a id='Objective'></a>
Let us describe the **MPC** algorithm for the problem just described above.  
At the current state $\boldsymbol x_{k}$ :

(a) **MPC** solves an $N$-step lookahead  problem: $\min _{\boldsymbol u_{k + i}: i=0, \ldots, N-1} \left(\sum_{i=0}^{N-1} \rho \left(\hat{\boldsymbol x}_{k + i}, \boldsymbol u_{k + i}\right)\right)$

(b) If $\left\{\boldsymbol u^{*}_{k}, \ldots, \boldsymbol u^{*}_{k+N-1}\right\}$ is the optimal control sequence of this problem, **MPC** applies $\boldsymbol u^{*}_{k}$ and discards the other controls $\boldsymbol u^{*}_{k+1}, \ldots, \boldsymbol u^{*}_{k+N-1}$. 

(c) At the next stage, **MPC** repeats this process, once the next state $\boldsymbol x_{k}$ is revealed.


<img src="MPC.svg" width=35% height=35% />

***

In [ ]:
%%capture
"""
Just importing all the necessary stuff here.
DO NOT CHANGE
"""
%matplotlib notebook
%load_ext autoreload
%autoreload 2

from rcognita_framework.pipelines.pipeline_3wrobot_NI import Pipeline3WRobotNI
from rcognita_framework.rcognita.actors import Actor
from rcognita_framework.rcognita.critics import CriticActionValue
from rcognita_framework.rcognita.models import ModelNN, ModelWeightContainer
from rcognita_framework.rcognita.scenarios import EpisodicScenarioBase
from rcognita_framework.rcognita import models
from rcognita_framework.rcognita.utilities import rc
from rcognita_framework.rcognita.optimizers import SciPyOptimizer, TorchOptimizer
import numpy as np
from torch import nn
import torch
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import warnings

In the following, for testing, we will use a system representing a three-wheeled robot described by a system of equations
$$\begin{cases}
\dot{x}_c=v \cos \alpha \\
\dot{y}_c=v \sin \alpha \\
\dot{\alpha}=\omega
\end{cases}$$ where $x_c$ and $y_c$ are coordinates of the center of mass, $v$ and $\omega$ are velocity of the center of mass and angular velocity respectively and these are components of the control $\boldsymbol u := (v, \omega)$ as well.  
The final visualization will look like the following picture:
<img src="robot.png" width=60% height=60% />

## Main goal:
Main goal here is to impement an RQL algorithm to combine both just described model-based and model-free approaches.

<h2 style="color:#A7BD3F;"> Section 1: Actor RQL implementation </h2>

Rollout Q-learning (RQL) actor optimizes the following actor objective:
            $J^a \left( y_k| \{u\}_k^{N_a} \right) = \sum_{i=0}^{N_a-1} \gamma^i r(y_{k+i}, u_{k+i}) + \gamma^{N_a} Q(y_{k+N_a}, u_{k+N_a}) = \text{MPC_objective} + \gamma^{N_a}\cdot\text{critic}$
            
Implement this objective in the `objective` method of `Actor`
            
**A brief recall**:

* `self.discount_factor` - that's how you can obtain $\gamma$ inside critic and actor
* `self.running_objective(observation, action)` - that's how you can obtain a value $r(\boldsymbol x, \boldsymbol u)$


In [ ]:
class ActorRQLStudent(Actor):
    
    def objective(
        self, action_sequence, observation,
    ):

        action_sequence_reshaped = rc.reshape(
            action_sequence, [self.prediction_horizon + 1, self.dim_input]
        ) ## use this as action sequence

        observation_sequence = [observation]

        observation_sequence_predicted = self.predictor.predict_sequence(
            observation, action_sequence_reshaped
        )

        observation_sequence = rc.vstack(
            (
                rc.reshape(observation, [1, self.dim_output]),
                observation_sequence_predicted,
            )
        ) ## use this as observation sequence

        actor_objective = 0
        
        #############################################
        # YOUR CODE BELOW
        #############################################

        
        #############################################
        # YOUR CODE ABOVE
        #############################################
        return actor_objective
    
    def update(self, observation, constraint_functions=(), time=None):
        """
        Method to update the current action or weight tensor.
        The old (previous) action or weight tensor is stored.
        The `time` argument is used for debugging purposes.
        """
        constraints = ()
        action_sequence_old = rc.rep_mat(
            self.action_old, 1, self.prediction_horizon + 1
        )

        action_sequence_init_reshaped = rc.reshape(
            action_sequence_old, [(self.prediction_horizon + 1) * self.dim_input,],
        )

        

        actor_objective = rc.func_to_lambda_with_params(
            self.objective, observation, var_prototype=action_sequence_init_reshaped
        )


        action_sequence_optimized = self.optimizer.optimize(
            actor_objective,
            action_sequence_init_reshaped,
            self.action_bounds,
            constraints=constraints,
        )

        self.model.update_and_cache_weights(action_sequence_optimized[: self.dim_input])
        self.action_old = self.model.cache.weights
        self.action = self.model.weights
        
        

<h2 style="color:#A7BD3F;"> Section 2: Critic model implementation </h2>

This task technically doesn't somehow differ from the corresponding task from the previous assignment. Choose sign-definiteness of your critic wisely.

In [ ]:
class ModelNNStudent(ModelNN):

    model_name = "NN"

    def __init__(self, dim_observation, dim_action, *args, weights = None, **kwargs):
        super().__init__(*args, **kwargs)
        
        #############################################
        # YOUR CODE BELOW
        #############################################

        #############################################
        # YOUR CODE ABOVE
        #############################################
        
        if weights is not None:
            self.load_state_dict(weights)
        self.double()
        self.cache_weights()

    def forward(self, input_tensor, weights=None):
        if weights is not None:
            self.update(weights)
        
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

        return x

<h2 style="color:#A7BD3F;"> Section 3: Testing</h2>

***

In [ ]:
class Pipeline3WRobotNIStudent(Pipeline3WRobotNI):

    def initialize_models(self):
        self.actor_model = models.ModelWeightContainer(weights_init=self.action_init)
        self.critic_model = ModelNNStudent(self.dim_output, self.dim_input)
        self.model_running_objective = models.ModelQuadForm(weights=self.R1)

        
    def initialize_optimizers(self):

        opt_options_torch = {"lr": self.critic_learning_rate}
        opt_options_scipy = {
            "maxiter": 1250,
            "maxfev": 5000,
            "disp": False,
            "adaptive": True,
            "xatol": 1e-7,
            "fatol": 1e-7,
        }
    
        self.actor_optimizer = SciPyOptimizer(
            opt_method="SLSQP", opt_options=opt_options_scipy
        )
        self.critic_optimizer = TorchOptimizer(opt_options_torch)

    def initialize_actor_critic(self):
        self.critic = CriticActionValue(
            dim_input=self.dim_input,
            dim_output=self.dim_output,
            data_buffer_size=self.data_buffer_size,
            running_objective=self.running_objective,
            discount_factor=self.discount_factor,
            optimizer=self.critic_optimizer,
            model=self.critic_model,
            sampling_time=self.sampling_time,
        )
        self.actor = ActorRQLStudent(
            self.prediction_horizon,
            self.dim_input,
            self.dim_output,
            self.control_mode,
            self.action_bounds,
            action_init=self.action_init,
            predictor=self.predictor,
            optimizer=self.actor_optimizer,
            critic=self.critic,
            running_objective=self.running_objective,
            model=self.actor_model,
        )
        
    def initialize_scenario(self):
        self.scenario = EpisodicScenarioBase(
            system=self.system,
            simulator=self.simulator,
            controller=self.controller,
            actor=self.actor,
            critic=self.critic,
            logger=self.logger,
            datafiles=self.datafiles,
            time_final=self.time_final,
            running_objective=self.running_objective,
            no_print=self.no_print,
            is_log=self.is_log,
            is_playback=self.is_playback,
            N_episodes=self.N_episodes,
            N_iterations=self.N_iterations,
            state_init=self.state_init,
            action_init=self.action_init,
        )

    def execute_pipeline(self, **kwargs):
        """
        Full execution routine
        """
        np.random.seed(42)
        super().execute_pipeline(**kwargs)
    
##### Execution here!!! Full list of kwargs can be seen at 
##### rcognita_framework.pipelines.config_blueprints in the ConfigInvertedPendulum

########### Play around here, tune lr and then go to the grading part.
pipeline = Pipeline3WRobotNIStudent()
pipeline.execute_pipeline(
    no_visual=True, 
    is_playback=False, # Don't touch
    critic_learning_rate= ,# Set your lr here
    no_print=False,
    time_final=10, 
    N_episodes=1, 
    N_iterations=1, 
    prediction_horizon=3,
    sampling_time=0.02, # Do not change it!
)

#### Grading.

In [ ]:
pipeline = Pipeline3WRobotNIStudent()
pipeline.execute_pipeline(
    no_visual=True, 
    is_playback=False, 
    critic_learning_rate= ,# Set your lr here
    no_print=True,
    time_final=10, # Don't touch
    N_episodes=1, 
    N_iterations=1, 
    prediction_horizon=3,
    sampling_time=0.02, # Do not change it!
)
import numpy as np
trajectory = np.array(pipeline.scenario.trajectory)
grade = np.clip(-1.5 * np.sum(trajectory[2500:, :3]) + 303, 0, 100)
print(f"Your final grade is {grade}")

### A set of questions. Leave your answers in this cell.
##### Set your prediction_horizon in the playground cell very small (mb even nulify it) and launch. What do you see? How do you explain it?
##### Answer:

##### Set your prediction_horizon bigger than 5 in the playground cell and launch . What do you see? How do you explain it?
##### Answer:

##### Set sampling_time to 0.1 in the playground cell and launch. What do you see? How do you explain it? Try to experiment with prediction_horizon now.
##### Answer: